# Poverty Rates for Cities Over 1M
- Joey Ashcroft Code Portion to be merged to master

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

In [2]:
# Census & gmaps API Keys
from config import (api_key, gkey)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [3]:
name = "?get=NAME,"
population = "B01003_001E"
poverty_count = "B17001_002E"
county = "for=county:*"

base_url = "https://api.census.gov/data/2017/acs/acs5"
query_url = base_url + name + population + "&" + poverty_count + "&" + county + "&key=" + api_key
query_url

'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01003_001E&B17001_002E&for=county:*&key=e4802ddb92bf7b297f5f10de4176b9d5341c10c8'

In [4]:
census_response = requests.get(query_url)
census_response = census_response.json()

# Convert to DataFrame
poverty_data = pd.DataFrame(census_response)

#replace header with row 1
new_header = poverty_data.iloc[0] #grab the first row for the header
poverty_data = poverty_data[1:] #take the data less the header row
poverty_data.columns = new_header #set the header row as the df header

# Column Reordering
poverty_data = poverty_data.rename(columns={"B01003_001E": "Population",
                                            "B17001_002E": "Poverty Count",
                                            "NAME": "Name", 
                                            "state": "State",
                                            "county":"County"})

# Add in Poverty Rate (Poverty Count / Population)
poverty_data["Poverty Rate"] = 100 * poverty_data["Poverty Count"].astype(int) / poverty_data["Population"].astype(int)

poverty_data.head()

,Name,Population,Poverty Count,State,County,Poverty Rate
1,"Corozal Municipio, Puerto Rico",34933,19146,72,047,54.807775
2,"Maunabo Municipio, Puerto Rico",11297,5945,72,095,52.624591
3,"Peñuelas Municipio, Puerto Rico",21661,12379,72,111,57.148793
4,"Ponce Municipio, Puerto Rico",148863,75187,72,113,50.507514
5,"San Sebastián Municipio, Puerto Rico",38970,21472,72,131,55.098794


In [5]:
poverty_data["County"] = poverty_data["Name"].str.split(pat=",", expand=True)[0]
poverty_data["State"] = poverty_data["Name"].str.split(pat=",", expand=True)[1]
# Parse Final DataFrame
poverty_data = poverty_data[["County","State","Population","Poverty Rate"]]
poverty_data.head()

,County,State,Population,Poverty Rate
1,Corozal Municipio,Puerto Rico,34933,54.807775
2,Maunabo Municipio,Puerto Rico,11297,52.624591
3,Peñuelas Municipio,Puerto Rico,21661,57.148793
4,Ponce Municipio,Puerto Rico,148863,50.507514
5,San Sebastián Municipio,Puerto Rico,38970,55.098794


### Geocode to get lat/long centroids of counties

In [71]:
# Target county
target_county = poverty_data["County"]
state = poverty_data['State']
coordinates=[]

for county, state in zip(target_county, state):
    # Build the endpoint URL
    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?address={county}&key={gkey}')
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()

    try:
        coordinates.append([county,state,geo_data['results'][0]['geometry']['location']])
        print(f"{county} County Coordinates Found: {geo_data['results'][0]['geometry']['location']}... Adding to List")
        
    except IndexError:
        print(f"{county} County not found... Skipping")

Corozal Municipio County Coordinates Found: {'lat': 18.3410622, 'lng': -66.3168357}... Adding to List
Maunabo Municipio County Coordinates Found: {'lat': 18.0071885, 'lng': -65.8993289}... Adding to List
Peñuelas Municipio County Coordinates Found: {'lat': 18.0633577, 'lng': -66.7273896}... Adding to List
Ponce Municipio County Coordinates Found: {'lat': 18.0110768, 'lng': -66.6140616}... Adding to List
San Sebastián Municipio County Coordinates Found: {'lat': 18.3354756, 'lng': -66.9946787}... Adding to List
Toa Alta Municipio County Coordinates Found: {'lat': 18.3882823, 'lng': -66.2482237}... Adding to List
Toa Baja Municipio County Coordinates Found: {'lat': 18.3882823, 'lng': -66.2482237}... Adding to List
Vieques Municipio County Coordinates Found: {'lat': 18.1474625, 'lng': -65.4448811}... Adding to List
Yabucoa Municipio County Coordinates Found: {'lat': 18.0505201, 'lng': -65.8793288}... Adding to List
Aguada Municipio County Coordinates Found: {'lat': 18.3801579, 'lng': -67.1

Marion County County Coordinates Found: {'lat': 44.8446393, 'lng': -122.5927411}... Adding to List
Pickens County County Coordinates Found: {'lat': 34.9046217, 'lng': -82.6483442}... Adding to List
Sumter County County Coordinates Found: {'lat': 28.6747526, 'lng': -82.08429009999999}... Adding to List
Jefferson County County Coordinates Found: {'lat': 39.5800298, 'lng': -105.2662931}... Adding to List
Choctaw County County Coordinates Found: {'lat': 33.9851472, 'lng': -95.51973939999999}... Adding to List
Franklin County County Coordinates Found: {'lat': 40.0154745, 'lng': -82.99316069999999}... Adding to List
Marengo County County Coordinates Found: {'lat': 32.2459085, 'lng': -87.77633329999999}... Adding to List
Russell County County Coordinates Found: {'lat': 32.3552497, 'lng': -85.1894045}... Adding to List
Cherokee County County Coordinates Found: {'lat': 34.2514526, 'lng': -84.4802606}... Adding to List
Covington County County Coordinates Found: {'lat': 31.3066438, 'lng': -86.499

Haines Borough County Coordinates Found: {'lat': 59.1392237, 'lng': -135}... Adding to List
Juneau City and Borough County Coordinates Found: {'lat': 58.3019444, 'lng': -134.4197221}... Adding to List
Lake and Peninsula Borough County Coordinates Found: {'lat': 58.0458356, 'lng': -156.5982286}... Adding to List
Southeast Fairbanks Census Area County Coordinates Found: {'lat': 63.9603869, 'lng': -143.6296999}... Adding to List
Yukon-Koyukuk Census Area County Coordinates Found: {'lat': 65.8443666, 'lng': -153.4302993}... Adding to List
Anchorage Municipality County Coordinates Found: {'lat': 61.2180556, 'lng': -149.9002778}... Adding to List
Aleutians East Borough County Coordinates Found: {'lat': 55.53880059999999, 'lng': -161.8988535}... Adding to List
Prince of Wales-Hyder Census Area County Coordinates Found: {'lat': 55.3242244, 'lng': -132.6044581}... Adding to List
Wrangell City and Borough County Coordinates Found: {'lat': 56.4708333, 'lng': -132.3766666}... Adding to List
Hoonah

Baxter County County Coordinates Found: {'lat': 36.3261815, 'lng': -92.38136209999999}... Adding to List
Ashley County County Coordinates Found: {'lat': 33.1565939, 'lng': -91.7538817}... Adding to List
Craighead County County Coordinates Found: {'lat': 35.8260959, 'lng': -90.6773046}... Adding to List
Lafayette County County Coordinates Found: {'lat': 39.0031288, 'lng': -93.9878427}... Adding to List
Perry County County Coordinates Found: {'lat': 40.371376, 'lng': -77.2405153}... Adding to List
Yell County County Coordinates Found: {'lat': 35.0810824, 'lng': -93.33889169999999}... Adding to List
Phillips County County Coordinates Found: {'lat': 34.4611206, 'lng': -90.86751339999999}... Adding to List
Poinsett County County Coordinates Found: {'lat': 35.6078586, 'lng': -90.6393702}... Adding to List
Sevier County County Coordinates Found: {'lat': 35.8271682, 'lng': -83.5070203}... Adding to List
Calhoun County County Coordinates Found: {'lat': 42.2073905, 'lng': -85.02334599999999}... 

Adams County County Coordinates Found: {'lat': 39.8398269, 'lng': -104.1930918}... Adding to List
Conejos County County Coordinates Found: {'lat': 37.2689711, 'lng': -106.2522143}... Adding to List
Custer County County Coordinates Found: {'lat': 38.0615578, 'lng': -105.3131185}... Adding to List
Douglas County County Coordinates Found: {'lat': 39.2587106, 'lng': -104.9388853}... Adding to List
Eagle County County Coordinates Found: {'lat': 39.576406, 'lng': -106.7234639}... Adding to List
Lake County County Coordinates Found: {'lat': 42.3689363, 'lng': -87.8271534}... Adding to List
La Plata County County Coordinates Found: {'lat': 37.2125857, 'lng': -107.763621}... Adding to List
Moffat County County Coordinates Found: {'lat': 40.6138379, 'lng': -108.2377519}... Adding to List
Pitkin County County Coordinates Found: {'lat': 39.159101, 'lng': -106.8178377}... Adding to List
Rio Grande County County Coordinates Found: {'lat': 37.5749508, 'lng': -106.4405872}... Adding to List
Cheyenne C

...

Kane County County Coordinates Found: {'lat': 41.987884, 'lng': -88.4016041}... Adding to List
Summit County County Coordinates Found: {'lat': 41.0918991, 'lng': -81.5596427}... Adding to List
Grand County County Coordinates Found: {'lat': 40.0727382, 'lng': -106.0640179}... Adding to List
Piute County County Coordinates Found: {'lat': 38.3997408, 'lng': -112.1430215}... Adding to List
Rich County County Coordinates Found: {'lat': 41.5629707, 'lng': -111.1891151}... Adding to List
Wasatch County County Coordinates Found: {'lat': 40.3629418, 'lng': -110.9983515}... Adding to List
Beaver County County Coordinates Found: {'lat': 40.6367812, 'lng': -80.365865}... Adding to List
Box Elder County County Coordinates Found: {'lat': 41.5380087, 'lng': -113.1918021}... Adding to List
Iron County County Coordinates Found: {'lat': 37.7854715, 'lng': -113.1918021}... Adding to List
Sanpete County County Coordinates Found: {'lat': 39.3090085, 'lng': -111.5706786}... Adding to List
Sevier County Coun

Newport News city County Coordinates Found: {'lat': 37.0870821, 'lng': -76.4730122}... Adding to List
Frederick County County Coordinates Found: {'lat': 39.3844507, 'lng': -77.4701972}... Adding to List
Stafford County County Coordinates Found: {'lat': 38.4334566, 'lng': -77.4242972}... Adding to List
Lynchburg city County Coordinates Found: {'lat': 37.4137536, 'lng': -79.14224639999999}... Adding to List
York County County Coordinates Found: {'lat': 34.9871367, 'lng': -81.2518833}... Adding to List
Richmond city County Coordinates Found: {'lat': 37.5407246, 'lng': -77.4360481}... Adding to List
Martinsville city County Coordinates Found: {'lat': 36.6915262, 'lng': -79.8725386}... Adding to List
Nelson County County Coordinates Found: {'lat': 37.7478599, 'lng': -78.8835548}... Adding to List
Essex County County Coordinates Found: {'lat': 40.7947466, 'lng': -74.2648829}... Adding to List
Pittsylvania County County Coordinates Found: {'lat': 36.7440186, 'lng': -79.4703885}... Adding to L

Southampton County County Coordinates Found: {'lat': 36.6788512, 'lng': -77.10249019999999}... Adding to List
Highland County County Coordinates Found: {'lat': 39.2228606, 'lng': -83.5070203}... Adding to List
Brunswick County County Coordinates Found: {'lat': 34.0275398, 'lng': -78.2020387}... Adding to List
Caroline County County Coordinates Found: {'lat': 38.0445557, 'lng': -77.2864879}... Adding to List
Gloucester County County Coordinates Found: {'lat': 39.7103023, 'lng': -75.1078332}... Adding to List
Northampton County County Coordinates Found: {'lat': 40.7749283, 'lng': -75.294569}... Adding to List
Patrick County County Coordinates Found: {'lat': 36.6886252, 'lng': -80.32131059999999}... Adding to List
Powhatan County County Coordinates Found: {'lat': 37.5643209, 'lng': -77.88245959999999}... Adding to List
Rockbridge County County Coordinates Found: {'lat': 37.8062214, 'lng': -79.5153773}... Adding to List
Shenandoah County County Coordinates Found: {'lat': 38.8413778, 'lng':

Kitsap County County Coordinates Found: {'lat': 47.6476607, 'lng': -122.6412583}... Adding to List
King County County Coordinates Found: {'lat': 47.5480339, 'lng': -121.9836029}... Adding to List
Pacific County County Coordinates Found: {'lat': 46.5180824, 'lng': -123.8264512}... Adding to List
Pend Oreille County County Coordinates Found: {'lat': 48.43590649999999, 'lng': -117.3173375}... Adding to List
San Juan County County Coordinates Found: {'lat': 48.61180350000001, 'lng': -122.9602621}... Adding to List
Skamania County County Coordinates Found: {'lat': 45.9428524, 'lng': -121.8962257}... Adding to List
Whitman County County Coordinates Found: {'lat': 46.8363403, 'lng': -117.5887295}... Adding to List
Yakima County County Coordinates Found: {'lat': 46.543606, 'lng': -120.7558291}... Adding to List
Clallam County County Coordinates Found: {'lat': 48.0405122, 'lng': -124.016766}... Adding to List
Cowlitz County County Coordinates Found: {'lat': 46.1746472, 'lng': -122.7746902}... A

Langlade County County Coordinates Found: {'lat': 45.220083, 'lng': -89.01021329999999}... Adding to List
Menominee County County Coordinates Found: {'lat': 45.4995277, 'lng': -87.5746277}... Adding to List
Outagamie County County Coordinates Found: {'lat': 44.4636545, 'lng': -88.4016041}... Adding to List
Vernon County County Coordinates Found: {'lat': 43.5576968, 'lng': -90.8294002}... Adding to List
Buffalo County County Coordinates Found: {'lat': 44.3102354, 'lng': -91.7538817}... Adding to List
Columbia County County Coordinates Found: {'lat': 33.5098514, 'lng': -82.25832969999999}... Adding to List
Kewaunee County County Coordinates Found: {'lat': 44.5202599, 'lng': -87.38107269999999}... Adding to List
La Crosse County County Coordinates Found: {'lat': 43.8617023, 'lng': -91.13530200000001}... Adding to List
Brown County County Coordinates Found: {'lat': 44.45616740000001, 'lng': -87.9334803}... Adding to List
Eau Claire County County Coordinates Found: {'lat': 44.7683909, 'lng'

In [87]:
#converting dictionary inside a list to a data series
coordinates = pd.DataFrame(coordinates)

county_coordinates = coordinates[2].apply(pd.Series)
county_coordinates['County'] = coordinates[0]
county_coordinates['State'] = coordinates[1]
county_coordinates = county_coordinates[['County','State','lat','lng']]
county_coordinates.head()

,County,State,lat,lng
0,Corozal Municipio,Puerto Rico,18.341062,-66.316836
1,Maunabo Municipio,Puerto Rico,18.007189,-65.899329
2,Peñuelas Municipio,Puerto Rico,18.063358,-66.727390
3,Ponce Municipio,Puerto Rico,18.011077,-66.614062
4,San Sebastián Municipio,Puerto Rico,18.335476,-66.994679


In [88]:
#export to csv
county_coordinates.to_csv("county_coordinates.csv", index=False)

In [17]:
# Run a request to endpoint and convert result to json
geo_data = requests.get(target_url).json()


# Print the json
print(geo_data)

{'results': [{'address_components': [{'long_name': 'Douglas County', 'short_name': 'Douglas County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Colorado', 'short_name': 'CO', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Douglas County, CO, USA', 'geometry': {'bounds': {'northeast': {'lat': 39.5662499, 'lng': -104.6605159}, 'southwest': {'lat': 39.1294379, 'lng': -105.3292199}}, 'location': {'lat': 39.2587106, 'lng': -104.9388853}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 39.5662499, 'lng': -104.6605159}, 'southwest': {'lat': 39.1294379, 'lng': -105.3292199}}}, 'place_id': 'ChIJxyXdfv2-bIcRWOQrqaUIXjI', 'types': ['administrative_area_level_2', 'political']}], 'status': 'OK'}


In [27]:
geo_data['results'][0]['geometry']['location']

{'lat': 39.2587106, 'lng': -104.9388853}

In [15]:
# Print the json (pretty printed)
print(json.dumps(geo_data, indent=4, sort_keys=True))

NameError: name 'json' is not defined

In [11]:
# Extract latitude and longitude
lat = geo_data["results"][0]["geometry"]["location"]["lat"]
lng = geo_data["results"][0]["geometry"]["location"]["lng"]

# Print the latitude and longitude
print('''
    City: {0}
    Latitude: {1}
    Longitude: {2}
    '''.format(target_city, lat, lng))


    City: Boise, Idaho
    Latitude: 43.6150186
    Longitude: -116.2023137
    
